In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
#import pandas as pd
import argparse
import os
import shutil
from pprint import pprint
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
doTraining = True
HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 32
NUM_EPOCHS = 10
TRAIN_VAL_DIR = '/content/drive/MyDrive/UniPassau/multimedia_retrieval/covid19_classification/data/COVID-19_Radiography_Dataset'
TEST_DIR = '/content/drive/MyDrive/UniPassau/multimedia_retrieval/covid19_classification/data/test'

In [14]:
classCnt = 0
imgPerClass = {}
for root, dirs, _ in os.walk(TRAIN_VAL_DIR):
    for name in dirs:
        print (os.path.join(root, name))

iterInputDir = iter(os.walk(TRAIN_VAL_DIR))
next(iterInputDir)
for (dirpath, dirnames, filenames) in iterInputDir:
    print('class folder {} has {} images'.format(os.path.basename(dirpath), len(filenames)))
    imgPerClass[classCnt] = len(filenames)
    classCnt += 1

if not os.path.exists(TEST_DIR):
    os.makedirs(TEST_DIR)

    for (dirpath, dirnames, filenames) in iterInputDir:
        # print('--------------- at iteration {}, dirpath {}: -----------------'.format(cnt, dirpath))
        filenames.sort()
        testClassPath = '{}/{}'.format(TEST_DIR, os.path.basename(dirpath))
        os.makedirs(testClassPath)
        for i in range(len(filenames)//10): 
            origImgPath = '{}/{}'.format(dirpath, filenames[i])
            testImgPath = '{}/{}'.format(testClassPath, filenames[i])
            shutil.move(origImgPath, testImgPath)
    
classWeights = {} 
largestClass = max(imgPerClass, key=imgPerClass.get)
for classKey, classValue in imgPerClass.items():
    classWeights[classKey] = imgPerClass[largestClass] / classValue

/content/drive/MyDrive/UniPassau/multimedia_retrieval/covid19_classification/data/COVID-19_Radiography_Dataset/COVID
/content/drive/MyDrive/UniPassau/multimedia_retrieval/covid19_classification/data/COVID-19_Radiography_Dataset/Normal
/content/drive/MyDrive/UniPassau/multimedia_retrieval/covid19_classification/data/COVID-19_Radiography_Dataset/Viral Pneumonia
class folder COVID has 3616 images
class folder Normal has 10192 images
class folder Viral Pneumonia has 1345 images


In [6]:
train_valid_datagen = ImageDataGenerator(#rescale=1./255,
                                        #  shear_range = 0.2,
                                        #  zoom_range = 0.2,
                                        #  horizontal_flip=True,
                                        #  preprocessing_function= preprocess_input,
                                        validation_split=0.1,
                                        preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

train_generator = train_valid_datagen.flow_from_directory(TRAIN_VAL_DIR,
                                                        subset="training",
                                                        batch_size=BATCH_SIZE,
                                                        seed=42,
                                                        shuffle=True,
                                                        class_mode="categorical",
                                                        target_size=(HEIGHT,WIDTH))

valid_generator = train_valid_datagen.flow_from_directory(TRAIN_VAL_DIR,
                                                        subset="validation",
                                                        batch_size=BATCH_SIZE,
                                                        seed=42,
                                                        shuffle=True,
                                                        class_mode="categorical",
                                                        target_size=(HEIGHT,WIDTH))

label_map = (train_generator.class_indices)
print(label_map)

Found 13639 images belonging to 3 classes.
Found 1514 images belonging to 3 classes.
{'COVID': 0, 'Normal': 1, 'Viral Pneumonia': 2}


In [8]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [9]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))

_, classNames, _ = next(os.walk(TRAIN_VAL_DIR))
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, dropout=dropout, fc_layers=FC_LAYERS, num_classes=len(classNames))

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = '/content/drive/MyDrive/UniPassau/multimedia_retrieval/covid19_classification/model/'
checkpoint = ModelCheckpoint(model_dir+"save_at_{epoch}.h5", save_best_only=True, monitor='val_loss', mode='min')
callbacks_list = [checkpoint]

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

if (doTraining == True):
    finetune_model.fit(x=train_generator,
                      steps_per_epoch=STEP_SIZE_TRAIN,
                      validation_data=valid_generator,
                      validation_steps=STEP_SIZE_VALID,
                      callbacks=callbacks_list,
                       #shuffle=False,
                      class_weight=classWeights,
                      epochs=NUM_EPOCHS)
    # history =finetune_model.fit_generator(generator=train_generator,
    #                                     steps_per_epoch=STEP_SIZE_TRAIN,
    #                                     validation_data=valid_generator,
    #                                     validation_steps=STEP_SIZE_VALID,
    #                                     callbacks=callbacks_list,
    #                                     epochs=NUM_EPOCHS, 
    #                                     shuffle=False,
    #                                     class_weight=classWeights)
    #finetune_model.evaluate(generator=valid_generator, steps=STEP_SIZE_VALID)
    finetune_model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID)
#     finetune_model.save('/content/drive/MyDrive/UniPassau/multimedia_retrieval/animal_classfier')
# else:
#     finetune_model.keras.models.load_model('/content/drive/MyDrive/UniPassau/multimedia_retrieval/animal_classfier')

87916544/87910968 [==============================] - 1s 0us/step
Epoch 1/10
426/426 [==============================] - 5613s 13s/step - loss: 1.2923 - accuracy: 0.7119 - val_loss: 0.2586 - val_accuracy: 0.9129
Epoch 2/10
426/426 [==============================] - 101s 236ms/step - loss: 0.4369 - accuracy: 0.8905 - val_loss: 0.3873 - val_accuracy: 0.8471
Epoch 3/10
426/426 [==============================] - 88s 206ms/step - loss: 0.3260 - accuracy: 0.9162 - val_loss: 0.3256 - val_accuracy: 0.8903
Epoch 4/10
426/426 [==============================] - 87s 204ms/step - loss: 0.2515 - accuracy: 0.9397 - val_loss: 0.2155 - val_accuracy: 0.9309
Epoch 5/10
426/426 [==============================] - 92s 217ms/step - loss: 0.1935 - accuracy: 0.9515 - val_loss: 0.1977 - val_accuracy: 0.9315
Epoch 6/10
426/426 [==============================] - 95s 224ms/step - loss: 0.1466 - accuracy: 0.9650 - val_loss: 0.2175 - val_accuracy: 0.9348
Epoch 7/10
426/426 [==============================] - 88s 207ms/

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [10]:
finetune_model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.16561603546142578, 0.9514627456665039]